# CSCI 3202, Fall 2021

# November 8, 2021

# Lecture 31:  Markov Decision Processes (MDP)

In [ ]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

Suppose there is a betting game that has these rules. 

- You start with some amount of money.
- You can wager any amount in increments of 1, with the maximum amount permitted = however much money you currently have.
- You flip a fair coin. If heads, you win your wager and your money increases by that amount. If tails, you lose your wager and your money decreases by that amount.
- You start playing this game with 5. You will quit if you have at least 10 at any point, or if you lose all of your money.

In typical American-style roulette, there are 18 red and 18 black bins, but also 2 green ones (0 and 00). Thus, the odds of winning a color-based bet are $ 18/38 \approx 0.473 $. So this coin-flipping game is like version of roulette.

**Part A:** We will approach the problem of figuring out how to play this game by using **Markov Decision Processes**, which include the idea of **risk versus reward**.

Create a list of the possible states given these conditions:

1. The rules of the game dictate that you can never have negative money, so the lowest state is $s=0$. What about the upper limit? You will quit if you have more than 10. But what if you had 9, and then won your bet?

2. We will also create a list of all the terminal states. These include 0 and any states with value 10 or greater.

In [ ]:
states = list(range(0, 19))
terminal_states = [0]+list(range(10,19))

**Part B:** 
1. Define an `actions` function that takes an argument `s` for the current state, and returns a list of all the possible actions - or wager amounts - that we can make. You may assume that `terminal_states` is implicitly known, so does not need to be fed in as an argument, and that `[None]` is the appropriate set of actions available to a terminal state.

In [ ]:
def actions(s):
    
    if s in terminal_states:
        return [None]
    else:
        return list(range(1,s+1))



2. Define a `transition` function that takes arguments `s` for the current states and `a` for the current action and returns a list of tuples. The second element of each tuple is an "adjacent" state that can be reached from `s` by action `a`, and the first element is the probability of this transition occurring.

In [ ]:
def transition(s,a):
    
    if a is None:
        return [(0, s)]
    
    else:
        return [(0.5, s+a), (0.5, s-a)]

3. Define a `reward` function that takes an argument of `s` for the current state, and returns the reward for that state.

First, you'll need to choose the rewards for the terminal states.

- If you win \\$10 or more, then a natural choice is to return the amount of money that you have won. For example, `reward(11)=11`
- If you lose all \\$5 that you started with, then you might think to return 0 as the reward. But really, you have actually lost \\$5, so a better choice of reward is -5.

Then, we need to choose the default reward for being in a non-terminal state. The typical thing to do is incorporate a small negative reward, so that the agent is incentivized to find a winning state. To start, let's use -0.01 as a reward for being in a non-terminal state.

In [ ]:
def reward(s):
    if s == 0:
        return -5
    elif s >= 10:
        return s-5
    else:
        return 0.01

**Part C:** Let's code up **value iteration** to solve for the utilities of each state under an optimal policy. Use a discount factor of 0.999 to reflect the fact that we can play many iterations of this game quickly, so future rewards are not really all that far away.

In [ ]:
gamma = 0.999

Let's walk through a single iteration of value iteration before we tackle the real deal. First, we must initialize something to track the largest change in utility of states this iteration, and an initial `utility` for all states. We can use a dictionary for `utility`, to map states (keys) to their utility (values). Here, we initialize all states to have 0 utility initially. In the end, it won't matter what value we used for initialization.

In [ ]:
max_change = 0
utility_old = {s: 0 for s in states}

We need to iterate over all states `s`, but let's start by just calculating the maximum expected utility for just one state. Then, we can add a loop around our code. Arbitrarily, let's say we start with `s=2`.

In [ ]:
s=2

For each available action from state `s`, we need to know what states are possible to reach and what their probabilities are. We need output from `transition(s,a)` for each possible action from state `s`.

Here is one way we could get the set of next states and their probabilities, just for the action of betting 1.

In [ ]:
next_states = transition(s, actions(s)[0])
print(next_states)

If we want all of the next states and their probabilities under all of the actions possible from `s`, then we could use the following list comprehension:

In [ ]:
next_states = [transition(s,a) for a in actions(s)]
print(next_states)

`next_states` is a list-of-lists, where the first list corresponds to the original output that we had from our simpler call to `transition`, and the second list corresponds to the output of `transition(s,actions(s)[1])`.

Now we need to calculate the maximum expected utility among all the possible actions. We can initialize a new utility dictionary to hold the updated utilities for each state.

In [ ]:
utility_new = utility_old.copy()

Here, we can implement a nice little loop to find the best utility among the possible actions from state `s`. Complete the codes below to calculate the expected utility (`newsum`) for each set of probabilities and states in `next_state`. The use the Bellman Equation to calculate `utility_new[s]` and update `max_change`, if necessary.

In [ ]:
best_utility = -999

# Since next_state is a list of lists, we need to loop through
# each list
for k in range(len(next_states)):
    newsum = 0
    # Within each list, we are listing more things, let's loop through those too
    for j in range(len(next_states[k])):
        
        prob_s_prime_given_s_a = next_states[k][j][0]
        utility = utility_old[next_states[k][j][1]]
        newsum = newsum + prob_s_prime_given_s_a*utility
    
    # Choose the maximum sum. This has a chance to be updated each time
    # the outer k loop is iterated through.
    best_utility = max(best_utility, newsum)

# Bellman update
utility_new[s] = reward(s)+gamma*best_utility
max_change = max(max_change, abs(utility_new[s]-utility_old[s]))

In [ ]:
print(utility_new[s])

Below is gently modified pseudocode from the lecture slides. We now have expressions for most of the things we need to fill in now.

When we actually iterate, we will need some convergence/exit criterion. That's where the tolerance `tol` and Equation 17.8 from the textbook come into play. 

$$\text{If}~~ |U_{i+1} - U_i| < \texttt{tol}\cdot \dfrac{1-\gamma}{\gamma},~~\text{then}~~ |U_{i+1} - U| < \texttt{tol}$$

What this tells us in plain English is that if some measure of difference between utilities on iteration $i$ and $i+1$ is less than $\text{tol}\cdot \dfrac{1-\gamma}{\gamma}$, then the error in the utility estimate in iteration $i+1$ is less than $\text{tol}$ itself.

Let's use an absolute error tolerance in utility of 0.001, and we can use `max_change` that we have been tracking as an estimate of $|U_{i+1} - U_i|$.  So we should exit our main iteration loop if $\texttt{max_change} <  \texttt{tol}\cdot \dfrac{1-\gamma}{\gamma}$

In [ ]:
# VALUE ITERATION:

gamma = 0.999
tol = 0.001

# initilize utility for all states

# iterate:

    # make a copy of current utility estimate, to be modified

    # initialize maximum change to 0

    # for each state s:

        # for each available action, what next states
        # are possible, and their probabilities?

        # calculate the maximum expected utility

        # new utility of s = reward(s) + 
        #                    discounted max expected utility

        # update maximum change in utilities, if needed

    # if maximum change in utility from one iteration to the
    # next is less than some tolerance, break!
        
# upon exit, utility_new is the utility of each state under an optimal policy

In [ ]:
# VALUE ITERATION:

gamma = 0.999
tol = 0.001

# initilize utility for all states

# iterate:

    # make a copy of current utility estimate, to be modified

    # initialize maximum change to 0

    # for each state s:

        # for each available action, what next states
        # are possible, and their probabilities?

        # calculate the maximum expected utility

        # new utility of s = reward(s) + 
        #                    discounted max expected utility

        # update maximum change in utilities, if needed

    # if maximum change in utility from one iteration to the
    # next is less than some tolerance, break!
        
# upon exit, utility_new is the utility of each state under an optimal policy

In [ ]:
# Solution:

# VALUE ITERATION:
df = 0.999
tol = 0.001

# initilize utility for all states
utility_new = {s : 0 for s in states}

# iterate:
while True:

    # make a copy of current utility estimate, to be modified
    utility_old = utility_new.copy()

    # initialize maximum change to 0
    max_change = 0

    # for each state s:
    for s in states:

        # for each available action, what next states
        # are possible, and their probabilities?
        next_states = [transition(s, a) for a in actions(s)]

        # calculate the maximum expected utility
        best_utility = -999
        for k in range(len(next_states)):
            newsum = sum([next_states[k][j][0]*utility_old[next_states[k][j][1]] for j in range(len(next_states[k]))])
            best_utility = max(best_utility, newsum)
            if len(next_states)==1:
                best_utility = newsum
        
        # new utility of s = reward(s) + 
        #                    discounted max expected utility
        utility_new[s] = reward(s) + df*best_utility

        # update maximum change in utilities, if needed
        max_change = max(max_change, abs(utility_new[s]-utility_old[s]))

    # if maximum change in utility from one iteration to the
    # next is less than some tolerance, break!
    if (df==1 and max_change < tol) or max_change < tol*(1-df)/df:
        break

# upon exit, utility_new is the utility of each state under an optimal policy
#utility_new

Finally, we need to compare the expected utility of all of the different actions available, so that we can choose the best one for each state:

$$\pi^*(s) = \underset{a \in \text{actions}(s)}{\arg\max} \sum_{s'} P(s' \mid s,a) U(s')$$

where $P$ we can get from the `transition` function, and $U$ is the utility we just found from value iteration.



In [ ]:
# initialize the policy for each state
policy = {s : None for s in states}

# loop over states to find the action that maximizes expected utility
for s in states:
    
    # initialize the best utility to something very bad, so we can improve it
    best_utility = (-999, None)
    
    # loop over actions, find which gives the highest expected utility
    for a in actions(s):
        
        # calculate the expected utility of action a from state s
        newsum = sum([p*utility_new[s2] for p, s2 in transition(s,a)])
        
        # if this action has higher expected utility than the current best,
        # replace the best (utility, action) tuple with this one
        if newsum > best_utility[0]:
            best_utility = (newsum, a)
            
    # now we have the action (second element) that leads
    # to the highest expected utility (first element)
    policy[s] = best_utility[1]

# upon exit, policy has the optimal policy for each state
policy